In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv("../input/pubg-finish-placement-prediction/train_V2.csv")
test = pd.read_csv("../input/pubg-finish-placement-prediction/test_V2.csv")
gender_submission = pd.read_csv("../input/pubg-finish-placement-prediction/sample_submission_V2.csv")

In [ ]:
data = pd.concat([train, test], sort = True)
print(len(train), len(test), len(data))
#print(data.isnull().sum())

In [ ]:
#data_form = pd.DataFrame(data)
#data_form.dtypes

In [ ]:
delete_columns = ['Id', 'groupId', 'matchId']
data.drop(delete_columns, axis = 1, inplace = True)

In [ ]:
data['fightRange'] = (data['kills'] + data['DBNOs']) / data['walkDistance'] * 100
data['fightRange'].replace([np.inf,-np.inf], np.nan, inplace = True)
data['fightRange'].fillna(0, inplace = True)

data['surpriseAttackRatio'] = (data['kills'] + data['DBNOs']) / data['damageDealt'] * 100
data['surpriseAttackRatio'].replace([np.inf,-np.inf], np.nan, inplace = True)
data['surpriseAttackRatio'].fillna(0, inplace = True)

In [ ]:
defaultmatch = ['squad-fpp', 'duo', 'solo-fpp', 'squad', 'duo-fpp', 'solo',
                'normal-squad-fpp', 'normal-solo-fpp','normal-duo-fpp',
                'normal-duo', 'normal-squad', 'normal-solo']
crashmatch = ['crashfpp', 'crashtpp']
flarematch = ['flarefpp', 'flaretpp']

data['defaultMatch'] = data['matchType'].replace(defaultmatch, 1)
data['defaultMatch'].replace(crashmatch, 0, inplace=True)
data['defaultMatch'].replace(flarematch, 0, inplace=True)

data['crashMatch'] = data['matchType'].replace(crashmatch, 1)
data['crashMatch'].replace(defaultmatch, 0, inplace=True)
data['crashMatch'].replace(flarematch, 0, inplace=True)

data['flareMatch'] = data['matchType'].replace(flarematch, 1)
data['flareMatch'].replace(defaultmatch, 0, inplace=True)
data['flareMatch'].replace(crashmatch, 0, inplace=True)

In [ ]:
data.drop('matchType', axis = 1, inplace = True)

In [ ]:
train = data[:len(train)]
test = data[len(train):]
train = train.dropna()
y_test_pred = 0

In [ ]:
winner_train = train['winPlacePerc']
train_valid_pred = winner_train * 0
train_source = train.drop('winPlacePerc', axis = 1)
test_source = test.drop('winPlacePerc', axis = 1)

学習

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import KFold

https://www.kaggle.com/aharless/lightgbm-cv-lb-282 In[9]を参考に

In [ ]:
params = {'task' : 'train',
          'boosting_type' : 'gbdt',
          'objective' : 'regression',
          'metric' : {'l2'},
          'num_leaves' : 31,
          'learning_rate' : 0.1,
          'feature_fraction' : 0.9,
          'bagging_fraction' : 0.8,
          'bagging_freq': 5,
          'verbose' : 0,
          }

In [ ]:
max_rounds = 1200
num_Fold = 5
kf = KFold(n_splits = num_Fold, random_state = 1, shuffle = True)

for i, (train_index,val_index) in enumerate(kf.split(train_source)):
    
    # Create data for this fold
    y_train = winner_train.iloc[train_index].copy()
    X_train = train_source.iloc[train_index,:].copy()
    test_set = test_source.copy()
    print( "Fold", i)

    # Run model for this fold
    fit_model = lgb.train(params,
                          train_set = lgb.Dataset(X_train, y_train),
                          num_boost_round = max_rounds,
                          verbose_eval = 50)
    test_pred = fit_model.predict(test_set)
    print("test_pred_head:", test_pred[:4], "\n")
    
    # Accumulate test set predictions
    y_test_pred += test_pred
    
print("Train is Done.")

# Average test set predictions
y_test_pred /= num_Fold
print("Predict_Avg:", y_test_pred)

In [ ]:
sub = pd.DataFrame(gender_submission['Id'])
sub['winPlacePerc'] = list(map(float, y_test_pred))
sub.to_csv("submission.csv", index = False)